In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable


# Image Modality Encoder
class ImageVAEEncoder(nn.Module):
    def __init__(self, input_channels=1, latent_dim=256):
        super(ImageVAEEncoder, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 16, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1)
        self.flatten = nn.Flatten()
        self.fc_mu = nn.Linear(in_features=64 * 28 * 28, out_features=latent_dim)
        self.fc_logvar = nn.Linear(in_features=64 * 28 * 28, out_features=latent_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = self.flatten(x)
        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)
        return mu, logvar
    
    

# Image Modality Decoder
class ImageVAEDecoder(nn.Module):
    def __init__(self, latent_dim=256, output_channels=1):
        super(ImageVAEDecoder, self).__init__()
        self.fc = nn.Linear(in_features=latent_dim, out_features=64 * 28 * 28)
        self.convtrans1 = nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.convtrans2 = nn.ConvTranspose2d(32, 16, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.convtrans3 = nn.ConvTranspose2d(16, output_channels, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.relu = nn.ReLU()
        self.output_activation = nn.Sigmoid()

    def forward(self, z):
        z = self.fc(z)
        z = z.view(-1, 64, 28, 28)
        z = self.relu(self.convtrans1(z))
        z = self.relu(self.convtrans2(z))
        z = self.output_activation(self.convtrans3(z))
        return z

# ECG Modality Encoder
class ECGVAEEncoder(nn.Module):
    def __init__(self, input_dim=60000, latent_dim=256):
        super(ECGVAEEncoder, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv1d(32, 64, kernel_size=3, stride=2, padding=1)
        self.flatten = nn.Flatten()
        self.fc_mu = nn.Linear(in_features=64 * (input_dim // 8), out_features=latent_dim)  # Adjusted for stride=2, 3 layers
        self.fc_logvar = nn.Linear(in_features=64 * (input_dim // 8), out_features=latent_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = self.flatten(x)
        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)
        return mu, logvar

# ECG Modality Decoder
class ECGVAEDecoder(nn.Module):
    def __init__(self, latent_dim=256, output_dim=60000):
        super(ECGVAEDecoder, self).__init__()
        self.fc = nn.Linear(in_features=latent_dim, out_features=64 * (output_dim // 8))
        self.convtrans1 = nn.ConvTranspose1d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.convtrans2 = nn.ConvTranspose1d(32, 16, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.convtrans3 = nn.ConvTranspose1d(16, 1, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.relu = nn.ReLU()
        self.output_activation = nn.Identity()  # Suitable for standardized data

    def forward(self, z):
        z = self.fc(z)
        z = z.view(-1, 64, z.size(1) // 64)  # Adjust the reshape for proper dimensions
        z = self.relu(self.convtrans1(z))
        z = self.relu(self.convtrans2(z))
        z = self.output_activation(self.convtrans3(z))
        return z

class MVAE(nn.Module):
    def __init__(self, image_encoder, ecg_encoder, image_decoder, ecg_decoder, latent_dim=256, num_classes=2):
        super(MVAE, self).__init__()
        self.image_encoder = image_encoder
        self.ecg_encoder = ecg_encoder
        self.image_decoder = image_decoder
        self.ecg_decoder = ecg_decoder

        # Classification layer
        self.classifier = nn.Sequential(
            nn.Linear(latent_dim * 2, 512),  # Latent dimensions from both encoders are concatenated
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, xray, ecg):
        xray_mu, xray_logvar = self.image_encoder(xray)
        ecg_mu, ecg_logvar = self.ecg_encoder(ecg)

        # Sample from the latent spaces
        xray_z = self.reparameterize(xray_mu, xray_logvar)
        ecg_z = self.reparameterize(ecg_mu, ecg_logvar)

        # Concatenate the latent representations
        combined_z = torch.cat([xray_z, ecg_z], dim=1)

        # Classification
        prediction = self.classifier(combined_z)

        # Optionally, decode the combined representation back into the original modalities
        reconstructed_xray = self.image_decoder(xray_z)
        reconstructed_ecg = self.ecg_decoder(ecg_z)

        return prediction, reconstructed_xray, reconstructed_ecg

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std


In [3]:
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torchvision import transforms
from PIL import Image
import os
import pydicom
import torch
from torch.utils.data import Dataset


class XRayDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = []
        self.labels = []

        # Loop through each label directory
        for label in [0, 1]:
            label_dir = os.path.join(root_dir, f'processed_label_{label}')
            for folder_name in os.listdir(label_dir):
                folder_path = os.path.join(label_dir, folder_name)
                image_name = os.listdir(folder_path)[0]  # Assuming only one image per folder
                if image_name.endswith('.jpg'):
                    self.images.append(os.path.join(folder_path, image_name))
                    self.labels.append(label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = self.images[idx]
        image = Image.open(image_path)
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label
    
 
# Initialize your dataset
xray_dataset = XRayDataset(root_dir='D:/Aspire_xray/xray', transform=transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
    transforms.ToTensor(),
]))


def process_dicom(file_path, sampling_rate=500):
    desired_length = 10 * sampling_rate  # 10 seconds of data
    try:
        dicom_data = pydicom.dcmread(file_path)
        if "WaveformSequence" in dicom_data:
            rhythm_waveform = dicom_data.WaveformSequence[1]
            wave_data = rhythm_waveform.get("WaveformData")
            num_channels = rhythm_waveform.NumberOfWaveformChannels
            wave_array = np.frombuffer(wave_data, dtype=np.int16)
            num_samples_per_channel = wave_array.size // num_channels
            
            if wave_array.size % num_channels == 0:
                wave_array = wave_array.reshape(num_samples_per_channel, num_channels)
                

                # Trim or Pad the array to 10 seconds
                if wave_array.shape[0] > desired_length:
                    wave_array = wave_array[:desired_length, :]
                elif wave_array.shape[0] < desired_length:
                    padding = np.zeros((desired_length - wave_array.shape[0], num_channels), dtype=wave_array.dtype)
                    wave_array = np.vstack((wave_array, padding))
                
                # Normalize the array
                wave_array = (wave_array - np.mean(wave_array, axis=0)) / np.std(wave_array, axis=0)

                return wave_array
            else:
                print(f"Unexpected data size in {file_path}. Skipping file.")
                return None
        else:
            print(f"No Waveform data found in {file_path}. Skipping file.")
            return None
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        return None


class ECGDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.ecg_data = []
        self.labels = []

        # Loop through each label directory
        for label in [0, 1]:
            label_dir = os.path.join(root_dir, f'processed_label_{label}')
            for folder_name in os.listdir(label_dir):
                folder_path = os.path.join(label_dir, folder_name)
                for file_name in os.listdir(folder_path):
                    if file_name.endswith('.dcm'):
                        file_path = os.path.join(folder_path, file_name)
                        ecg_waveform = process_dicom(file_path)
                        if ecg_waveform is not None:
                            self.ecg_data.append(ecg_waveform)
                            self.labels.append(label)

    def __len__(self):
        return len(self.ecg_data)

    def __getitem__(self, idx):
        ecg_waveform = self.ecg_data[idx]
        label = self.labels[idx]
        # Reshape waveform to [1, signal_length]
        ecg_waveform = ecg_waveform.reshape(1, -1)  
        return torch.tensor(ecg_waveform, dtype=torch.float32), label

# Usage example
ecg_dataset = ECGDataset(root_dir='D:/Aspire_ecg/ecg')

class CombinedDataset(Dataset):
    def __init__(self, xray_dataset, ecg_dataset):
        self.xray_dataset = xray_dataset
        self.ecg_dataset = ecg_dataset
        assert len(xray_dataset) == len(ecg_dataset), "Datasets must be of the same length."
        
        # Assuming the labels are the same for both datasets and can be directly accessed
        self.labels = [label for _, label in xray_dataset]

    def __len__(self):
        return len(self.xray_dataset)

    def __getitem__(self, idx):
        xray_image, xray_label = self.xray_dataset[idx]
        ecg_waveform, ecg_label = self.ecg_dataset[idx]
        
        # Ensure the labels match if they are supposed to be the same
        assert xray_label == ecg_label, "Labels do not match for the same index."
        
        return xray_image, ecg_waveform, xray_label  # Use either xray_label or ecg_label

    def get_labels(self):
        return self.labels

# Instantiate the combined dataset
combined_dataset = CombinedDataset(xray_dataset, ecg_dataset)
labels = combined_dataset.get_labels()


In [9]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score

import random 
import numpy as np
def set_seed(seed_value):
    """Set seed for reproducibility."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed_value)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set a seed value
seed = 42
set_seed(seed)

# Model instantiation for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

latent_dim = 256
num_classes = 2  # Assuming binary classification for PAWP prediction

image_encoder = ImageVAEEncoder(input_channels=1, latent_dim=latent_dim).to(device)
ecg_encoder = ECGVAEEncoder(input_dim=60000, latent_dim=latent_dim).to(device)
image_decoder = ImageVAEDecoder(latent_dim=latent_dim, output_channels=1).to(device)
ecg_decoder = ECGVAEDecoder(latent_dim=latent_dim, output_dim=60000).to(device)

classification_loss_fn = nn.CrossEntropyLoss()

def train_and_evaluate(model, train_loader, val_loader, optimizer, loss_fn, num_epochs=10):
    best_auc = 0
    for epoch in range(num_epochs):
        model.train()
        for xray_images, ecg_waveforms, labels in train_loader:
            xray_images = xray_images.to(device)
            ecg_waveforms = ecg_waveforms.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            predictions, _, _ = model(xray_images, ecg_waveforms)
            loss = loss_fn(predictions, labels)
            loss.backward()
            optimizer.step()
        
        # Evaluation phase
        model.eval()
        all_predictions = []
        all_labels = []
        with torch.no_grad():
            for xray_images, ecg_waveforms, labels in val_loader:
                xray_images = xray_images.to(device)
                ecg_waveforms = ecg_waveforms.to(device)
                
                predictions, _, _ = model(xray_images, ecg_waveforms)
                all_predictions.append(predictions.cpu().numpy())
                all_labels.append(labels.numpy())  # Labels are used on CPU for metric calculation
        
        all_predictions = np.concatenate(all_predictions, axis=0)
        all_labels = np.concatenate(all_labels, axis=0)
        
        # Convert softmax outputs to binary predictions for AUC calculation
        binary_predictions = all_predictions[:, 1]  # Assuming class 1 is the positive class
        
        # Calculate AUC and accuracy
        auc = roc_auc_score(all_labels, binary_predictions)
        accuracy = accuracy_score(all_labels, np.argmax(all_predictions, axis=1))
        
        if auc > best_auc:
            best_auc = auc
            # Save the best model, adjust as needed
    
    return auc, accuracy

# Assuming combined_dataset is defined
skf = StratifiedKFold(n_splits=10)
dataset_size = len(combined_dataset)
indices = np.arange(dataset_size)
labels = np.array(combined_dataset.get_labels())

auc_scores = []
accuracy_scores = []

for train_index, test_index in skf.split(indices, labels):
    train_subset = Subset(combined_dataset, train_index)
    val_subset = Subset(combined_dataset, test_index)
    
    train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=32, shuffle=False)
    
    model = MVAE(image_encoder, ecg_encoder, image_decoder, ecg_decoder, latent_dim=256, num_classes=num_classes).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.CrossEntropyLoss()
    
    auc, accuracy = train_and_evaluate(model, train_loader, val_loader, optimizer, loss_fn, num_epochs=10)
    auc_scores.append(auc)
    accuracy_scores.append(accuracy)

mean_auc = np.mean(auc_scores)
std_auc = np.std(auc_scores)
mean_accuracy = np.mean(accuracy_scores)
std_accuracy = np.std(accuracy_scores)

print(f"AUC: {mean_auc:.4f} ± {std_auc:.4f}")
print(f"Accuracy: {mean_accuracy:.4f} ± {std_accuracy:.4f}")


AUC: 0.7372 ± 0.0554
Accuracy: 0.7249 ± 0.0457
